<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="fig/cover-small.jpg">

*本文摘自 Jake VanderPlas 的 [Python 之旅](http://www.oreilly.com/programming/free/a-whirlwind-tour-of-python.csp)，内容可在 [GitHub](https://github.com/jakevdp/WhirlwindTourOfPython) 上找到。*

*文本和代码在 [CC0](https://github.com/jakevdp/WhirlwindTourOfPython/blob/master/LICENSE) 许可下发布；另见配套项目，[Python 数据科学手册](https://github.com/jakevdp/PythonDataScienceHandbook)。*

*中文翻译由 [ZhangCongke](https://ckeyzhang.github.io/) 提供，项目可在 [GitHub](https://github.com/CKeyZhang/WhirlwindTourOfPython-CN) 上找到。*

<!--NAVIGATION-->
< [定义和使用函数](08-Defining-Functions.ipynb) | [目录](Index.ipynb) | [迭代器](10-Iterators.ipynb) >

# 错误和异常

无论你作为程序员的技能如何，你总会犯编程错误。
这些错误主要有三种类型：

- *语法错误*：代码不是有效的 Python（通常很容易修复）
- *运行时错误*：语法有效的代码未能执行，可能是因为用户输入无效（有时很容易修复）
- *语义错误*：逻辑错误：代码执行没有问题，但结果并非你所期望的（往往很难追踪和修复）

这里我们将专注于如何干净利落地处理 *运行时错误*。
正如我们将看到的，Python 通过其 *异常处理* 框架来处理运行时错误。

## 运行时错误

如果你在 Python 中做过任何编程，你可能遇到过运行时错误。
它们可能以多种方式发生。

例如，如果你尝试引用一个未定义的变量：

In [2]:
print(Q)

NameError: name 'Q' is not defined

或者尝试一个未定义的操作：

In [3]:
1 + 'abc'

TypeError: unsupported operand type(s) for +: 'int' and 'str'

或者你可能试图计算一个数学上无定义的结果：

In [4]:
2 / 0

ZeroDivisionError: division by zero

或者你可能试图访问一个不存在的序列元素：

In [5]:
L = [1, 2, 3]
L[1000]

IndexError: list index out of range

请注意，在每种情况下，Python 都足够友好，不仅表明发生了错误，还抛出了一个 *有意义的* 异常，其中包含了关于到底出了什么问题以及错误发生的确切代码行的信息。
能够获得像这样的有意义的错误信息，在尝试追踪代码问题的根源时非常有用。

## 捕获异常：``try`` 和 ``except``
Python 为你处理运行时异常提供的主要工具是 ``try``...``except`` 子句。
其基本结构如下：

In [6]:
try:
    print("这将首先被执行")
except:
    print("只有在发生错误时才会执行这里")

这将首先被执行


请注意，第二个块在这里没有被执行：这是因为第一个块没有返回错误。
让我们在 ``try`` 块中放入一个有问题的语句，看看会发生什么：

In [7]:
try:
    print("让我们尝试一下：")
    x = 1 / 0 # ZeroDivisionError
except:
    print("出问题了！")

让我们尝试一下：
出问题了！


在这里，我们看到当 ``try`` 语句中抛出了错误（在这种情况下，是一个 ``ZeroDivisionError``），错误被捕获，然后执行了 ``except`` 语句。

这种用法通常用于在函数或代码的其他部分中检查用户输入。
例如，我们可能希望有一个函数，它捕获零除错误并返回一些其他值，比如一个足够大的数字，如 $10^{100}$：

In [8]:
def safe_divide(a, b):
    try:
        return a / b
    except:
        return 1E100

In [9]:
safe_divide(1, 2)

0.5

In [10]:
safe_divide(2, 0)

1e+100

这段代码有一个微妙的问题：当出现另一种类型的异常时会发生什么？例如，这可能不是我们想要的结果：

In [11]:
safe_divide (1, '2')

1e+100

将整数和字符串相除会引发一个 ``TypeError``，我们的过于急切的代码捕获了它，并假设它是 ``ZeroDivisionError``！
因此，几乎总是最好明确地捕获异常：

In [12]:
def safe_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return 1E100

In [13]:
safe_divide(1, 0)

1e+100

In [14]:
safe_divide(1, '2')

TypeError: unsupported operand type(s) for /: 'int' and 'str'

现在我们只捕获零除错误，让所有其他错误按原样通过。

## 抛出异常：``raise``
我们已经看到了在使用 Python 语言的某些部分时，有信息量的异常是多么有价值。
在你编写的代码中使用有信息量的异常同样有价值，这样代码的用户（首先是你自己！）可以找出导致错误的原因。

抛出自定义异常的方法是使用 ``raise`` 语句。例如：

In [15]:
raise RuntimeError("我的错误消息")

RuntimeError: 我的错误消息

作为这个可能有用的示例，让我们回到我们之前定义的 ``fibonacci`` 函数：

In [16]:
def fibonacci(N):
    L = []
    a, b = 0, 1
    while len(L) < N:
        a, b = b, a + b
        L.append(a)
    return L

这里的一个潜在问题是输入值可能是负数。
这在我们的函数中目前不会导致任何错误，但我们可以让用户知道不支持负的 ``N``。
按照惯例，由于无效的参数值导致的错误会导致抛出一个 ``ValueError``：

In [17]:
def fibonacci(N):
    if N < 0:
        raise ValueError("N 必须是非负数")
    L = []
    a, b = 0, 1
    while len(L) < N:
        a, b = b, a + b
        L.append(a)
    return L

In [18]:
fibonacci(10)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [19]:
fibonacci(-10)

ValueError: N 必须是非负数

现在用户确切地知道为什么输入无效，并且甚至可以使用 ``try``...``except`` 块来处理它！

In [20]:
N = -10
try:
    print("尝试这个...")
    print(fibonacci(N))
except ValueError:
    print("值无效：需要做些别的事情")

尝试这个...
值无效：需要做些别的事情


## 更深入地了解异常

简要地，我想在这里提及一些你可能会遇到的其他概念。
我不会深入讲解这些概念以及如何以及何时使用它们，而是仅仅向你展示语法，以便你可以自行进一步探索。

### 访问错误消息

有时在 ``try``...``except`` 语句中，你希望能够操作错误消息本身。
这可以通过 ``as`` 关键字来实现：

In [21]:
try:
    x = 1 / 0
except ZeroDivisionError as err:
    print("错误类别是：  ", type(err))
    print("错误消息是：", err)

错误类别是：   <class 'ZeroDivisionError'>
错误消息是： division by zero


通过这种模式，你可以进一步自定义函数的异常处理。

### 定义自定义异常
除了内置异常外，还可以通过 *类继承* 定义自定义异常。
例如，如果你想有一个特殊的 ``ValueError``，可以这样做：

In [22]:
class MySpecialError(ValueError):
    pass

raise MySpecialError("这是消息")

MySpecialError: 这是消息

这将允许你使用一个 ``try``...``except`` 块，只捕获这种类型的错误：

In [23]:
try:
    print("做些事情")
    raise MySpecialError("[详细错误消息]")
except MySpecialError:
    print("做些别的事情")

做些事情
做些别的事情


当你开发更定制化的代码时，你可能会发现这很有用。

## ``try``...``except``...``else``...``finally``
除了 ``try`` 和 ``except`` 外，你还可以使用 ``else`` 和 ``finally`` 关键字来进一步调整代码对异常的处理。
基本结构如下：

In [24]:
try:
    print("在这里尝试一些事情")
except:
    print("只有失败时才会执行这里")
else:
    print("只有成功时才会执行这里")
finally:
    print("无论如何都会执行这里")

在这里尝试一些事情
只有成功时才会执行这里
无论如何都会执行这里


``else`` 的用途在这里很清楚，但 ``finally`` 的重点是什么？
好吧，``finally`` 子句确实 *无论如何* 都会执行：我通常看到它用于在操作完成后进行某种清理。

<!--NAVIGATION-->
< [定义和使用函数](08-Defining-Functions.ipynb) | [目录](Index.ipynb) | [迭代器](10-Iterators.ipynb) >